In [1]:
import pandas as pd
import numpy as np
import os, sys, json, logging
os.chdir('../')
logging.basicConfig(level=logging.DEBUG,
                    format='%(levelname)-9s: %(message)s', stream=sys.stdout)

In [2]:
for variant in range(8):
    variant = json.load(open('data/check/test_{}.json'.format(str(variant).zfill(2))))
    print(variant['tasks'][0]['text'])
    print()

Прочитайте текст и выполните следующее задание.(1) В некоторых моделях современных автомобилей раскрытие подушек безопасности при непристёгнутых ремнях автоматически блокируется.(2) Дело в том, что срабатывание подушек, происходящее с огромной скоростью, сродни сильному удару.(3) <...> тело непристёгнутого человека в момент аварии не будет зафиксировано ремнём и сместится в сторону, то подушка не только окажется плохим помощником, но и сама может стать причиной серьёзной травмы.Укажите два предложения, в которых верно передана главная информация, содержащаяся в тексте.Запишите номера этих предложений.

Прочитайте текст и выполните задание.(1) В отношении городского благоустройства Великий Новгород оставлял позади себя многие средневековые города Западной Европы.(2) В Западной Европе первые мостовые появляются лишь в конце XII века, а в Новгороде уже в XI веке деревом мостили все улицы, а также дворы внутри усадеб.(3) Кроме того, знаменитое Ярославово дворище в Новгороде имело первый в 

In [3]:
task = variant['tasks'][0]
task

{'id': '1',
 'meta': {'language': 'ru', 'source': 'yandex_test'},
 'text': 'Прочитайте текст и выполните задание.(1)Если над человеческим ухом пропоёт петух, человек на время оглохнет: мощность петушиного крика составляет 130 децибел – то же самое будет, если стоять в пятнадцати метрах от взлетающего реактивного самолёта.(2)Однако петухи не теряют слух от собственного пения, <…> в тот момент, когда петух открывает клюв для крика, наружный слуховой проход у него сужается примерно на четверть, а мягкие ткани наполовину прикрывают барабанную перепонку.(3)У кур слуховой проход устроен немного иначе, и «заткнуть уши» они не могут, но потерять слух от близкого соседства с петухами им не грозит: куры слушают петухов не вплотную, к тому же у птиц слуховые рецепторы довольно быстро регенерируются.Укажите варианты ответов, в которых верно передана ГЛАВНАЯ информация, содержащаяся в тексте.Запишите номера этих предложений.',
 'attachments': [],
 'question': {'type': 'multiple_choice',
  'min_choi

In [4]:
from src.solvers_utils import split_task_and_text
from src.bert_embedder import get_features

In [5]:
split_task_and_text(task['text'])

('Прочитайте текст и выполните задание.Укажите варианты ответов, в которых верно передана ГЛАВНАЯ информация, содержащаяся в тексте.Запишите номера этих предложений.',
 '(1)Если над человеческим ухом пропоёт петух, человек на время оглохнет: мощность петушиного крика составляет 130 децибел – то же самое будет, если стоять в пятнадцати метрах от взлетающего реактивного самолёта.(2)Однако петухи не теряют слух от собственного пения, <…> в тот момент, когда петух открывает клюв для крика, наружный слуховой проход у него сужается примерно на четверть, а мягкие ткани наполовину прикрывают барабанную перепонку.(3)У кур слуховой проход устроен немного иначе, и «заткнуть уши» они не могут, но потерять слух от близкого соседства с петухами им не грозит: куры слушают петухов не вплотную, к тому же у птиц слуховые рецепторы довольно быстро регенерируются')

In [6]:
from src.solvers_utils import split_task_and_text
from src.bert_embedder import get_features
from sklearn.metrics import pairwise_distances
import logging

def solver_1(task, emb_size=100, metric='cosine'):
    
    question, text = split_task_and_text(task['text'])
    
    if 'кажите два предложения' in question:
        n_answers = 2
    else:
        n_answers = 1
    logging.debug('N answers: {}'.format(n_answers))
    
    logging.disable(sys.maxsize)
    text_emb = get_features([text], emb_size)
    text_emb = np.mean(list(text_emb.values()), axis=0)
    
    answers_emb = [get_features([choice['text']], emb_size)
                   for choice in task['question']['choices']]
    answers_emb = np.array([np.mean(list(answer_emb.values()), axis=0)
                            for answer_emb in answers_emb])
    logging.disable(logging.NOTSET)
    
    dist = pairwise_distances([text_emb], answers_emb, metric)[0]
    logging.debug('Distances: {}'.format(dist))
    
    answer = np.argsort(dist)[:n_answers] + 1
    
    return answer.astype(str).tolist()

In [7]:
solver_1(task, 768, 'euclidean')

INFO     : Distances: [53.3433   56.700676 55.595757 54.265663 55.071503]


['1']

In [6]:
np.argsort([1, 0, 2, 10, -1])

array([4, 1, 0, 2, 3])

In [21]:
text_emb = get_features(['.'.join(task['text'].split('.')[1:-3])], 50)
text_emb = np.mean(list(text_emb.values()), axis=0)

W0914 02:43:42.323073 140047627564864 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f5f1324ab00>) includes params argument, but params are not passed to Estimator.
W0914 02:43:42.323776 140047627564864 estimator.py:1811] Using temporary folder as model directory: /tmp/tmpl_t8lppl
W0914 02:43:42.324451 140047627564864 tpu_context.py:211] eval_on_tpu ignored because use_tpu is False.


In [22]:
text_emb

array([-0.7282592 , -1.7589452 , -2.6455202 ,  3.7007298 ,  0.3510464 ,
        2.0399728 ,  0.49261966, -2.9653459 , -0.63507146, -1.9149581 ,
        1.0712415 , -0.36609668,  1.1525809 ,  1.2414777 ,  1.286407  ,
       -0.04015633, -0.12285427, -0.15983468, -0.31544533,  0.81810397,
       -1.0578545 , -1.7316316 ,  2.957862  , -0.82667357, -2.377799  ,
       -1.9033453 ,  2.1446664 ,  0.0354691 ,  0.97538346,  0.58440053,
       -0.8541069 ,  1.4011079 , -3.9944174 ,  0.9101914 ,  1.5156144 ,
        0.45890534,  0.47125992, -0.37775838, -0.5213252 , -1.4346595 ,
       -1.6102533 , -1.7767421 , -1.4029396 ,  1.0198892 ,  3.1617825 ,
       -0.30541503, -1.2078701 ,  1.3548846 ,  0.39209068, -2.1368845 ],
      dtype=float32)

In [15]:
answers_emb = [get_features([choice['text']], 50) for choice in task['question']['choices']]
answers_emb = np.array([np.mean(list(answer_emb.values()), axis=0) for answer_emb in answers_emb])

W0914 02:40:08.932776 140047627564864 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f5f1324ab00>) includes params argument, but params are not passed to Estimator.
W0914 02:40:08.934615 140047627564864 estimator.py:1811] Using temporary folder as model directory: /tmp/tmp6xjm5d2r
W0914 02:40:08.935340 140047627564864 tpu_context.py:211] eval_on_tpu ignored because use_tpu is False.
W0914 02:40:15.761628 140047627564864 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f5f0c480680>) includes params argument, but params are not passed to Estimator.
W0914 02:40:15.762543 140047627564864 estimator.py:1811] Using temporary folder as model directory: /tmp/tmpjie6hw73
W0914 02:40:15.763040 140047627564864 tpu_context.py:211] eval_on_tpu ignored because use_tpu is False.
W0914 02:40:22.486649 140047627564864 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f5f1324a

In [16]:
from sklearn.metrics import pairwise_distances

In [23]:
pairwise_distances([text_emb], answers_emb, 'cosine')

array([[1.1294763, 1.145634 , 0.62581  , 0.8356792, 0.8329265]],
      dtype=float32)

In [24]:
pairwise_distances([text_emb], answers_emb)

array([[15.19437 , 15.502262, 11.744391, 13.798402, 14.269027]],
      dtype=float32)

In [ ]:
../code/bert_embedder.py

In [7]:
model = build_model(configs.elmo_embedder.elmo_ru_wiki, download=True)

2019-09-13 18:48:36.527 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-wiki_600k_steps.tar.gz download because of matching hashes
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
Using TensorFlow backend.


In [25]:
model([['ABC', 'asdf'], ['asd']])

[array([[-0.07741089, -0.0929472 , -0.7708945 , ...,  0.4405897 ,
         -0.22633982, -0.5019179 ],
        [-0.8294297 , -1.0968993 , -0.43525547, ...,  0.04747637,
          0.7677173 , -0.14973152]], dtype=float32),
 array([[-1.4425784 , -0.6769587 , -0.43116167, ...,  1.2079057 ,
          0.6422209 , -0.5624919 ]], dtype=float32)]

In [38]:
len(emb)

10

In [10]:
from sklearn.neighbors import NearestNeighbors

In [17]:
from sklearn.metrics import pairwise_distances

In [48]:
text_emb = model(task['text'].split('.'))
text_emb = np.array([word.mean(0) for word in text_emb])

In [56]:
answers_emb = model([choice['text'] for choice in task['question']['choices']])
answers_emb = np.array([answer_emb.mean(0) for answer_emb in answers_emb])

In [50]:
text_emb.shape

(10, 2560)

In [57]:
answers_emb.shape

(5, 2560)

In [60]:
pairwise_distances([text_emb.mean(0)], answers_emb, 'cosine')

array([[0.56168747, 0.56239   , 0.5775629 , 0.52426624, 0.51112986]],
      dtype=float32)

In [11]:
nn_model = NearestNeighbors(1)

In [16]:
nn_model.fit(np.array(emb))

ValueError: setting an array element with a sequence.

In [42]:
!pip install --upgrade sklearn scipy numpy pandas

Requirement already up-to-date: sklearn in /opt/conda/lib/python3.6/site-packages (0.0)
    100% |████████████████████████████████| 25.2MB 1.4MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/e5/e6/c3fdc53aed9fa19d6ff3abf97dfad768ae3afce1b7431f7500000816bda5/numpy-1.17.2-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 10.5MB 4.5MB/s eta 0:00:01
Requirement not upgraded as not directly required: scikit-learn in /opt/conda/lib/python3.6/site-packages (from sklearn) (0.21.2)
Requirement not upgraded as not directly required: pytz>=2017.2 in /opt/conda/lib/python3.6/site-packages (from pandas) (2018.5)
    100% |████████████████████████████████| 235kB 39.0MB/s ta 0:00:01
Requirement not upgraded as not directly required: joblib>=0.11 in /opt/conda/lib/python3.6/site-packages (from scikit-learn->sklearn) (0.12.5)
Requirement not upgraded as not directly required: six>=1.5 in /opt/conda/lib/python3.6/site-packages (from python-dateutil>=2.6.